In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

In [2]:
path = glob.glob(r'csv_data\데이터 분석과제 (1)\*')
for i,j in enumerate(path):
    print(i,j)

0 csv_data\데이터 분석과제 (1)\GTW07-034_210801.csv
1 csv_data\데이터 분석과제 (1)\GTW07-034_210802.csv
2 csv_data\데이터 분석과제 (1)\GTW07-034_210803.csv
3 csv_data\데이터 분석과제 (1)\GTW07-034_210804.csv
4 csv_data\데이터 분석과제 (1)\GTW07-034_210805.csv
5 csv_data\데이터 분석과제 (1)\GTW07-034_210806.csv
6 csv_data\데이터 분석과제 (1)\GTW07-034_210807.csv
7 csv_data\데이터 분석과제 (1)\GTW07-034_210808.csv
8 csv_data\데이터 분석과제 (1)\GTW07-034_210809.csv
9 csv_data\데이터 분석과제 (1)\GTW07-034_210810.csv
10 csv_data\데이터 분석과제 (1)\GTW07-034_210811.csv
11 csv_data\데이터 분석과제 (1)\GTW07-034_210812.csv
12 csv_data\데이터 분석과제 (1)\GTW07-034_210813.csv
13 csv_data\데이터 분석과제 (1)\GTW07-034_210814.csv
14 csv_data\데이터 분석과제 (1)\GTW07-034_210815.csv
15 csv_data\데이터 분석과제 (1)\GTW07-034_210816.csv
16 csv_data\데이터 분석과제 (1)\GTW07-034_210817.csv
17 csv_data\데이터 분석과제 (1)\GTW07-034_210818.csv
18 csv_data\데이터 분석과제 (1)\GTW07-034_210819.csv
19 csv_data\데이터 분석과제 (1)\GTW07-034_210820.csv
20 csv_data\데이터 분석과제 (1)\GTW07-034_210821.csv
21 csv_data\데이터 분석과제 (1)\GTW07-034_210822.cs

In [ ]:
tmp_df =pd.read_csv(path[0]).iloc[:,:83]
for i in range(1, 31):
    tmp_df = tmp_df.append(pd.read_csv(path[i]).iloc[:,:83], ignore_index=True)
for i in range(33, 35):
    tmp_df = tmp_df.append(pd.read_csv(path[i]).iloc[:,:83], ignore_index=True)

In [ ]:
tmp_df

In [ ]:
df = tmp_df.dropna()
df

In [ ]:
# 각 column별 자료형 확인
for j,i in enumerate(list(df)):
    print(j, i, df[i].dtype)
# 몇몇 변수가 null값으로 인해 object 변수로 입력돼있음.

In [ ]:
# 사용하지 않는 변수 제거
df = df.drop(axis='columns', columns=[' X_PEAK', ' X_PEAK_ALARM', ' X_RMS', ' X_RMS_ALARM'
                                     , ' Y_PEAK', ' Y_PEAK_ALARM', ' Y_RMS', ' Y_RMS_ALARM'
                                     , ' Z_PEAK', ' Z_PEAK_ALARM', ' Z_RMS', ' Z_RMS_ALARM'
                                     , 'MEASR_YYYY', 'MEASR_MM', 'MEASR_DD', 'MEASR_HH'])
# null값으로 인해 object 변수로 입력된 것을 float으로 변환.
for i in range(11, 23, 2):
    df[list(df)[i]] = df[list(df)[i]].astype(float)
df

In [ ]:
# 자료형 다시 확인
for j,i in enumerate(list(df)):
    print(j, i, df[i].dtype)

In [ ]:
# MEASR_DTTM 컬럼을 시간 자료형으로 변경 후 시간 순으로 데이터 정렬
df['MEASR_DTTM'] = pd.to_datetime(df['MEASR_DTTM'])
df = df.sort_values(by=['MEASR_DTTM'])
df = df.reset_index(drop=True)
# 앞에 sensor_id 등 제거
df = df.iloc[:,6:]
df

In [ ]:
# 변수명 앞에 띄어쓰기 없애기
col = []
for i in range(len(list(df))):
    col.append(list(df)[i].split(' ')[-1])
df.columns= col

df

In [ ]:
# 2021-08-29  5:22:00 AM
# 2021-09-03  9:35:00 AM 전조 시점

In [ ]:
import datetime
df['STATUS'] = '정상' #  label column 생성
df.loc[(df['MEASR_DTTM']>= pd.to_datetime('2021-08-29  5:22:00'))&(df['MEASR_DTTM']<= pd.to_datetime('2021-09-03  9:35:00')),'STATUS']='전조'

In [ ]:
df

In [ ]:
df = pd.concat([df['MEASR_DTTM'], df.iloc[:,np.arange(1,60,2)], df['STATUS']], axis=1)
df.to_csv(r'data\data.csv')

## 여기부터

In [ ]:
import pandas as pd
df = pd.read_csv(r'data\data.csv',index_col=0)
df['MEASR_DTTM'] =pd.to_datetime(df['MEASR_DTTM'])

In [ ]:
from scipy import  stats
# 모평균 차이 검정

# t-test(정규성 만족 할 때)
for i in range(len(list(df))-2):
    print(list(df)[i+1],' ', end='')
    print(stats.ttest_ind(df[df['STATUS']=='정상'].iloc[:,i+1], df[df['STATUS']!='정상'].iloc[:,i+1], equal_var=True))
#     print(stats.ttest_ind(df_list[i][df_list[i]['Y']==1].iloc[:,-2], df_list[i][df_list[i]['Y']==0].iloc[:,-2], equal_var=True))


In [ ]:
# 모평균 차이 검정

# 정규성 만족 안하는 경우 (Mann-Whitney U test)
for i in range(len(list(df))-2):
    print(list(df)[i+1],' ', end='')
    print(stats.mannwhitneyu(df[df['STATUS']=='정상'].iloc[:,i+1], df[df['STATUS']!='정상'].iloc[:,i+1]))


In [ ]:
# 등분산 검정

def f_test(x, y):
    x = np.array(x)
    y = np.array(y)
    f = np.var(x, ddof=1) / np.var(y, ddof=1)  # calculate F test statistic
    dfn = x.size - 1  # define degrees of freedom numerator
    dfd = y.size - 1  # define degrees of freedom denominator
    p = 2 * min(stats.f.cdf(f, dfn, dfd), 1-stats.f.cdf(f, dfn, dfd)) # find p-value of F test statistic
    return f, p

In [ ]:
import numpy as np
# f-test
for i in range(len(list(df))-2):
    print(list(df)[i+1],' ', end='')
    print(f_test(df[df['STATUS']=='정상'].iloc[:,i+1], df[df['STATUS']!='정상'].iloc[:,i+1]))

In [ ]:
# Bartlett’s test, Fligner-Killeen test, Levene test
for i in range(len(list(df))-2):
    print(list(df)[i+1],' ', end='')
    a = df[df['STATUS']=='정상'].iloc[:,i+1]
    b = df[df['STATUS']!='정상'].iloc[:,i+1]
    print(stats.bartlett(a, b), stats.fligner(a, b), stats.levene(a, b), sep = '\n')
    print('\n')

In [ ]:
train_data = df[df['MEASR_DTTM']<pd.to_datetime('2021-08-27 00:00:00')]
test_data = df[df['MEASR_DTTM']>=pd.to_datetime('2021-08-27 00:00:00')]
train_data

In [ ]:
print(len(train_data), len(test_data))

# Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest
isol = IsolationForest()

In [ ]:
isol.fit(train_data.iloc[:,21:31])

In [ ]:
for i in range(len(list(df))):
    print(i, list(df)[i])

In [ ]:
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
isol_score = df[['MEASR_DTTM', 'STATUS']].copy()
isol_score['SCORE']= isol.score_samples(df.iloc[:,21:31])*-1
isol_score

In [ ]:
# threshold 설정
import numpy as np
nboot = 100
alpha=0.0001 # 백분위 기준 이상치가 1%되게 threshold 설정한다는 의미
threshold_series = isol_score[isol_score['MEASR_DTTM']<train_test_split_point]['SCORE']

bootsam = np.random.choice(threshold_series, size=len(threshold_series) * nboot, replace=True, p=None)
bootsam.shape = (nboot, len(threshold_series))

isol_threshold  = np.mean(np.percentile(bootsam, q=(1 - alpha) * 100, axis=1))
isol_threshold

In [ ]:
threshold = isol_threshold
# threshold =0.746
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
train_test_split_point2 = train_test_split_point.timestamp()*1000
import plotly.express as px
plot_df = isol_score.copy()


# threshold 아래부분 sampling해서 그래프 용량 줄이기
if len(plot_df) > 100000:
    tmp = plot_df[plot_df['SCORE'] < threshold].sort_values(by='SCORE', ascending = False).copy()
    tmp = tmp[:100000]
    plot_df = pd.concat((tmp, plot_df[plot_df['SCORE']>=threshold]))
    plot_df = plot_df.sort_values('MEASR_DTTM')


plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
fig.add_vline(x=train_test_split_point2
                  , line_dash="dash"
                 , annotation_text = 'Test 구간'
    #               , line_color="red"
                 )
fig.add_hrect(
    y0=threshold, 
    y1=np.min(plot_df['이상 점수']), line_width=0, 
    annotation_text = '정상구간',
    fillcolor="green", opacity=0.2)
fig.show()

tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
print('test set에서 false 경고: ', len(tmp), 'test set에서 true 경고: ', len(tmp2))
print('train set에서 false 경고: ', len(tmp3))

## 15분 내 3회경고시 알람

In [ ]:
import datetime
timedel = 15 # 몇분 이내에 3회이상 경고 떠야 알람 울릴 것인지
cutoff = 3 # 15분 이내에 몇개 이상일 때알람 울릴 것인지
alarm_df = isol_score.copy()
alarm_df['ALARM'] = False
for i in np.where(alarm_df['SCORE'] > threshold)[0]:
    tmp = alarm_df[(alarm_df['MEASR_DTTM']<alarm_df.iloc[i,0])&(alarm_df['MEASR_DTTM']>=alarm_df.iloc[i,0]-datetime.timedelta(minutes=timedel))]
    if len(tmp[tmp['SCORE']>threshold])>cutoff:
        alarm_df.iloc[i, -1] = True

tmp = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = alarm_df[(alarm_df['ALARM']==True)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]      
print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
print('train set에서 false 알람: ', len(tmp3))
start =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
if len(tmp2) >0:
    start = tmp2.iloc[0,0]
    print('가짜알람비율: ',len(tmp)/(len(tmp)+len(tmp2)))
else:
    print('알람없음.')
sstart =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
end = alarm_df[alarm_df['STATUS']=='전조'].iloc[-1,0]

print('최초알람시점 비율',(end-start)/(end-sstart))

## Y 만 사용한 isol

In [ ]:
isol2 = IsolationForest()
isol2.fit(train_data.iloc[:,11:21])
isol_score2 = df[['MEASR_DTTM', 'STATUS']].copy()
isol_score2['SCORE']= isol2.score_samples(df.iloc[:,11:21])*-1

In [ ]:
# threshold 설정
import numpy as np
nboot = 100
alpha=0.0001 # 백분위 기준 이상치가 1%되게 threshold 설정한다는 의미
threshold_series = isol_score2[isol_score2['MEASR_DTTM']<train_test_split_point]['SCORE']

bootsam = np.random.choice(threshold_series, size=len(threshold_series) * nboot, replace=True, p=None)
bootsam.shape = (nboot, len(threshold_series))

isol_threshold2  = np.mean(np.percentile(bootsam, q=(1 - alpha) * 100, axis=1))
isol_threshold2

In [ ]:
threshold = isol_threshold2
# threshold =0.74865
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
train_test_split_point2 = train_test_split_point.timestamp()*1000
import plotly.express as px
plot_df = isol_score2.copy()


# threshold 아래부분 sampling해서 그래프 용량 줄이기
if len(plot_df) > 100000:
    tmp = plot_df[plot_df['SCORE'] < threshold].sort_values(by='SCORE', ascending = False).copy()
    tmp = tmp[:100000]
    plot_df = pd.concat((tmp, plot_df[plot_df['SCORE']>=threshold]))
    plot_df = plot_df.sort_values('MEASR_DTTM')

plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
fig.add_vline(x=train_test_split_point2
                  , line_dash="dash"
                 , annotation_text = 'Test 구간'
    #               , line_color="red"
                 )
fig.add_hrect(
    y0=threshold, 
    y1=np.min(plot_df['이상 점수']), line_width=0, 
    annotation_text = '정상구간',
    fillcolor="green", opacity=0.2)
fig.show()

tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
print('test set에서 false 경고: ', len(tmp), 'test set에서 true 경고: ', len(tmp2))
print('train set에서 false 경고: ', len(tmp3))

In [ ]:
import datetime
timedel = 15 # 몇분 이내에 3회이상 경고 떠야 알람 울릴 것인지
cutoff = 3 # 15분 이내에 몇개 이상일 때알람 울릴 것인지
alarm_df = isol_score2.copy()
alarm_df['ALARM'] = False
for i in np.where(alarm_df['SCORE'] > threshold)[0]:
    tmp = alarm_df[(alarm_df['MEASR_DTTM']<alarm_df.iloc[i,0])&(alarm_df['MEASR_DTTM']>=alarm_df.iloc[i,0]-datetime.timedelta(minutes=timedel))]
    if len(tmp[tmp['SCORE']>threshold])>cutoff:
        alarm_df.iloc[i, -1] = True

tmp = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = alarm_df[(alarm_df['ALARM']==True)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]      
print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
print('train set에서 false 알람: ', len(tmp3))
start =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
if len(tmp2) >0:
    start = tmp2.iloc[0,0]
    print('가짜알람비율: ',len(tmp)/(len(tmp)+len(tmp2)))
else:
    print('알람없음.')
sstart =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
end = alarm_df[alarm_df['STATUS']=='전조'].iloc[-1,0]

print('최초알람시점 비율',(end-start)/(end-sstart))

## Auto Encoder

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
from tensorflow.keras import backend as K
checkpoint_path = 'best_weights.h5'

scaler = MinMaxScaler()
data = train_data.iloc[:,1:-1].copy()
data[:] = scaler.fit_transform(data[:])

shape = data.shape
x1 = Input(shape=(shape[1],))
x = Lambda(lambda x: K.expand_dims(x,axis=2))(x1)
x = Conv1D(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
x = Dropout(0.2)(x)
x = Conv1D(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
x = Conv1DTranspose(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
x = Dropout(0.2)(x)
x = Conv1DTranspose(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
x = Conv1DTranspose(filters=1, kernel_size=7, padding = 'same')(x)
x = Flatten()(x)
x2 = Dense(shape[1], activation='sigmoid')(x)

model = Model(x1, x2)

model.compile(optimizer='adam', loss='mse')

es = EarlyStopping(patience=10, mode='auto', monitor = 'val_loss')
mc = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True, verbose=1)


model.fit(data, data, epochs=100000, validation_split= 0.1, callbacks=[es, mc], batch_size=128, verbose=1) 
model.load_weights(checkpoint_path)


## 이미 오토인코더학습 했고 불러오기만 원할 때

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.models import Model
# from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
# from tensorflow.keras import backend as K
# checkpoint_path = 'best_weights.h5'

# scaler = MinMaxScaler()
# data = train_data.iloc[:,1:-1].copy()
# data[:] = scaler.fit_transform(data[:])

# shape = data.shape
# x1 = Input(shape=(shape[1],))
# x = Lambda(lambda x: K.expand_dims(x,axis=2))(x1)
# x = Conv1D(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1D(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1DTranspose(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=1, kernel_size=7, padding = 'same')(x)
# x = Flatten()(x)
# x2 = Dense(shape[1], activation='sigmoid')(x)

# model = Model(x1, x2)
# model.load_weights(checkpoint_path)

In [ ]:
auto_df = df.copy()
auto_df.iloc[:,1:-1] = scaler.transform(auto_df.iloc[:,1:-1])

In [ ]:
pred = model.predict(auto_df.iloc[:,1:-1])
score = np.mean(np.abs(pred - auto_df.iloc[:,1:-1]), axis=1)

auto_score = df[['MEASR_DTTM', 'STATUS']].copy()
auto_score['SCORE']= score

In [ ]:
# threshold 설정
import numpy as np
nboot = 100
alpha=0.0001 # 백분위 기준 이상치가 0.1%되게 threshold 설정한다는 의미
threshold_series = auto_score[auto_score['MEASR_DTTM']<train_test_split_point]['SCORE']

bootsam = np.random.choice(threshold_series, size=len(threshold_series) * nboot, replace=True, p=None)
bootsam.shape = (nboot, len(threshold_series))

auto_threshold  = np.mean(np.percentile(bootsam, q=(1 - alpha) * 100, axis=1))
auto_threshold

In [ ]:
threshold = auto_threshold
# threshold =0.12
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
train_test_split_point2 = train_test_split_point.timestamp()*1000
import plotly.express as px
plot_df = auto_score.copy()
# threshold 아래부분 sampling해서 그래프 용량 줄이기
if len(plot_df) > 100000:
    tmp = plot_df[plot_df['SCORE'] < threshold].sort_values(by='SCORE', ascending = False).copy()
    tmp = tmp[:100000]
    plot_df = pd.concat((tmp, plot_df[plot_df['SCORE']>=threshold]))
    plot_df = plot_df.sort_values('MEASR_DTTM')

plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
fig.add_vline(x=train_test_split_point2
                  , line_dash="dash"
                 , annotation_text = 'Test 구간'
    #               , line_color="red"
                 )
fig.add_hrect(
    y0=threshold, 
    y1=np.min(plot_df['이상 점수']), line_width=0, 
    annotation_text = '정상구간',
    fillcolor="green", opacity=0.2)
fig.show()
# test set에서 false 알람.
tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
print('test set에서 false 경고: ', len(tmp), 'test set에서 true 경고: ', len(tmp2))
print('train set에서 false 경고: ', len(tmp3))

## 15분 내 3번 경고시 알람

In [ ]:
import datetime
timedel = 15 # 몇분 이내에 3회이상 경고 떠야 알람 울릴 것인지
cutoff = 3 # 15분 이내에 몇개 이상일 때알람 울릴 것인지
alarm_df = auto_score.copy()
alarm_df['ALARM'] = False
for i in np.where(alarm_df['SCORE'] > threshold)[0]:
    tmp = alarm_df[(alarm_df['MEASR_DTTM']<alarm_df.iloc[i,0])&(alarm_df['MEASR_DTTM']>=alarm_df.iloc[i,0]-datetime.timedelta(minutes=timedel))]
    if len(tmp[tmp['SCORE']>threshold])>cutoff:
        alarm_df.iloc[i, -1] = True

tmp = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = alarm_df[(alarm_df['ALARM']==True)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]      
print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
print('train set에서 false 알람: ', len(tmp3))
start =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
if len(tmp2) >0:
    start = tmp2.iloc[0,0]
    print('가짜알람비율: ',len(tmp)/(len(tmp)+len(tmp2)))
else:
    print('알람없음.')
sstart =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
end = alarm_df[alarm_df['STATUS']=='전조'].iloc[-1,0]

print('최초알람시점 비율',(end-start)/(end-sstart))

## Y만 사용한 오토인코더

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
from tensorflow.keras import backend as K
checkpoint_path = 'best_weights_Y.h5'

scaler = MinMaxScaler()
data2 = train_data.iloc[:,11:21].copy()
data2[:] = scaler.fit_transform(data2[:])

shape = data2.shape
x1 = Input(shape=(shape[1],))
x = Lambda(lambda x: K.expand_dims(x,axis=2))(x1)
x = Conv1D(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
x = Dropout(0.2)(x)
x = Conv1D(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
x = Conv1DTranspose(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
x = Dropout(0.2)(x)
x = Conv1DTranspose(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
x = Conv1DTranspose(filters=1, kernel_size=7, padding = 'same')(x)
x = Flatten()(x)
x2 = Dense(shape[1], activation='sigmoid')(x)

model2 = Model(x1, x2)

model2.compile(optimizer='adam', loss='mse')

es = EarlyStopping(patience=10, mode='auto', monitor = 'val_loss')
mc = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True, verbose=1)


model2.fit(data2, data2, epochs=100000, validation_split= 0.1, callbacks=[es, mc], batch_size=128, verbose=1) 
model2.load_weights(checkpoint_path)


## 이미 오토인코더학습 했고 불러오기만 원할 때

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.models import Model
# from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
# from tensorflow.keras import backend as K
# checkpoint_path = 'best_weights_Y.h5'

# scaler = MinMaxScaler()
# data2 = train_data.iloc[:,11:21].copy()
# data2[:] = scaler.fit_transform(data2[:])

# shape = data2.shape
# x1 = Input(shape=(shape[1],))
# x = Lambda(lambda x: K.expand_dims(x,axis=2))(x1)
# x = Conv1D(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1D(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1DTranspose(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=1, kernel_size=7, padding = 'same')(x)
# x = Flatten()(x)
# x2 = Dense(shape[1], activation='sigmoid')(x)

# model2 = Model(x1, x2)
# model2.load_weights(checkpoint_path)

In [ ]:
auto_df2 = df.copy()
auto_df2.iloc[:,11:21] = scaler.transform(auto_df.iloc[:,11:21])

In [ ]:
pred = model2.predict(auto_df2.iloc[:,11:21])
score = np.mean(np.abs(pred - auto_df2.iloc[:,11:21]), axis=1)

auto_score2 = df[['MEASR_DTTM', 'STATUS']].copy()
auto_score2['SCORE']= score

In [ ]:
# threshold 설정
import numpy as np
nboot = 100
alpha=0.0001 # 백분위 기준 이상치가 0.1%되게 threshold 설정한다는 의미
threshold_series = auto_score2[auto_score2['MEASR_DTTM']<train_test_split_point]['SCORE']

bootsam = np.random.choice(threshold_series, size=len(threshold_series) * nboot, replace=True, p=None)
bootsam.shape = (nboot, len(threshold_series))

auto_threshold2  = np.mean(np.percentile(bootsam, q=(1 - alpha) * 100, axis=1))
auto_threshold2

In [ ]:
threshold = auto_threshold2
# threshold =0.12
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
train_test_split_point2 = train_test_split_point.timestamp()*1000
import plotly.express as px
plot_df = auto_score2.copy()
# threshold 아래부분 sampling해서 그래프 용량 줄이기
if len(plot_df) > 100000:
    tmp = plot_df[plot_df['SCORE'] < threshold].sort_values(by='SCORE', ascending = False).copy()
    tmp = tmp[:100000]
    plot_df = pd.concat((tmp, plot_df[plot_df['SCORE']>=threshold]))
    plot_df = plot_df.sort_values('MEASR_DTTM')

plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
fig.add_vline(x=train_test_split_point2
                  , line_dash="dash"
                 , annotation_text = 'Test 구간'
    #               , line_color="red"
                 )
fig.add_hrect(
    y0=threshold, 
    y1=np.min(plot_df['이상 점수']), line_width=0, 
    annotation_text = '정상구간',
    fillcolor="green", opacity=0.2)
fig.show()
# test set에서 false 알람.
tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
print('test set에서 false 경고: ', len(tmp), 'test set에서 true 경고: ', len(tmp2))
print('train set에서 false 경고: ', len(tmp3))

## 15분 내 3번 경고시 알람

In [ ]:
import datetime
timedel = 15 # 몇분 이내에 3회이상 경고 떠야 알람 울릴 것인지
cutoff = 3 # 15분 이내에 몇개 이상일 때알람 울릴 것인지
alarm_df = auto_score2.copy()
alarm_df['ALARM'] = False
for i in np.where(alarm_df['SCORE'] > threshold)[0]:
    tmp = alarm_df[(alarm_df['MEASR_DTTM']<alarm_df.iloc[i,0])&(alarm_df['MEASR_DTTM']>=alarm_df.iloc[i,0]-datetime.timedelta(minutes=timedel))]
    if len(tmp[tmp['SCORE']>threshold])>cutoff:
        alarm_df.iloc[i, -1] = True

tmp = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = alarm_df[(alarm_df['ALARM']==True)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]      
print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
print('train set에서 false 알람: ', len(tmp3))
start =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
if len(tmp2) >0:
    start = tmp2.iloc[0,0]
    print('가짜알람비율: ',len(tmp)/(len(tmp)+len(tmp2)))
else:
    print('알람없음.')
sstart =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
end = alarm_df[alarm_df['STATUS']=='전조'].iloc[-1,0]

print('최초알람시점 비율',(end-start)/(end-sstart))


# Y만 활용했을 때보다 X, Z 모두 활용했을 때 더강력해보임

In [ ]:
# # threshold =0.12
# train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
# train_test_split_point2 = train_test_split_point.timestamp()*1000
# import plotly.express as px
# plot_df = auto_score.copy()

# plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
# fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
# fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
# fig.add_vline(x=train_test_split_point2
#                   , line_dash="dash"
#                  , annotation_text = 'Test 구간'
#     #               , line_color="red"
#                  )
# fig.add_hrect(
#     y0=threshold, 
#     y1=np.min(plot_df['이상 점수']), line_width=0, 
#     annotation_text = '정상구간',
#     fillcolor="green", opacity=0.2)
# fig.show()
# # test set에서 false 알람.
# tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
# tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
# tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
# tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
# tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
# tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
# print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
# print('train set에서 false 알람: ', len(tmp3))

# 아래 두개는 용량이 커서  시행이 안됨

## SVM

In [ ]:
data

In [ ]:
from sklearn.svm import OneClassSVM
if len(data) > 100000:
    tmp =  np.random.choice(data.index, size = 100000)
    svm_data = data.loc[tmp,:].copy()
svm = OneClassSVM(max_iter = 10000).fit(svm_data) # auto encoder에서 사용한거 그대로 사용. / 데이터가 많아 kernel svm을 못사용함.

svm_score = df[['MEASR_DTTM', 'STATUS']].copy()
svm_score['SCORE']= svm.score_samples(auto_df.iloc[:,1:-1])*-1

In [ ]:
# threshold 설정
import numpy as np
nboot = 100
alpha=0.0001 # 백분위 기준 이상치가 0.1%되게 threshold 설정한다는 의미
threshold_series = svm_score[svm_score['MEASR_DTTM']<train_test_split_point]['SCORE']

bootsam = np.random.choice(threshold_series, size=len(threshold_series) * nboot, replace=True, p=None)
bootsam.shape = (nboot, len(threshold_series))

svm_threshold  = np.mean(np.percentile(bootsam, q=(1 - alpha) * 100, axis=1))
svm_threshold

In [ ]:
plot_df = svm_score.copy()
threshold = svm_threshold
# threshold =0.12
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
train_test_split_point2 = train_test_split_point.timestamp()*1000
import plotly.express as px

# threshold 아래부분 sampling해서 그래프 용량 줄이기
if len(plot_df) > 100000:
    tmp = plot_df[plot_df['SCORE'] < threshold].sort_values(by='SCORE', ascending = False).copy()
    tmp = tmp[:100000]
    plot_df = pd.concat((tmp, plot_df[plot_df['SCORE']>=threshold]))
    plot_df = plot_df.sort_values('MEASR_DTTM')

plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
fig.add_vline(x=train_test_split_point2
                  , line_dash="dash"
                 , annotation_text = 'Test 구간'
    #               , line_color="red"
                 )
fig.add_hrect(
    y0=threshold, 
    y1=np.min(plot_df['이상 점수']), line_width=0, 
    annotation_text = '정상구간',
    fillcolor="green", opacity=0.2)
fig.show()
# test set에서 false 알람.
tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
print('test set에서 false 경고: ', len(tmp), 'test set에서 true 경고: ', len(tmp2))
print('train set에서 false 경고: ', len(tmp3))

In [ ]:
# 전조구간에서 얼마나 빨리 울렸는지 : 최초 알람 발생 시점 (70시간 / 90시간 1에 가까울 수록 최초 알람 발생 시점이 빠른 measure)
# 전조구간에 얼마나 많이 울렸는지, 정상구간에 얼마나 적게 울렸는지 : false alarm rate : 18 / 747
# 전조구간에 얼마나 많이 울렸는지    

## 15분 내 3번 경고시 알람

In [ ]:
import datetime
timedel = 15 # 몇분 이내에 3회이상 경고 떠야 알람 울릴 것인지
cutoff = 3 # 15분 이내에 몇개 이상일 때알람 울릴 것인지
alarm_df = svm_score.copy()
alarm_df['ALARM'] = False
for i in np.where(alarm_df['SCORE'] > threshold)[0]:
    tmp = alarm_df[(alarm_df['MEASR_DTTM']<alarm_df.iloc[i,0])&(alarm_df['MEASR_DTTM']>=alarm_df.iloc[i,0]-datetime.timedelta(minutes=timedel))]
    if len(tmp[tmp['SCORE']>threshold])>cutoff:
        alarm_df.iloc[i, -1] = True

tmp = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = alarm_df[(alarm_df['ALARM']==True)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]   


print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
print('train set에서 false 알람: ', len(tmp3))
start =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
if len(tmp2) >0:
    start = tmp2.iloc[0,0]
    print('가짜알람비율: ',len(tmp)/(len(tmp)+len(tmp2)))
else:
    print('알람없음.')
sstart =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
end = alarm_df[alarm_df['STATUS']=='전조'].iloc[-1,0]

print('최초알람시점 비율',(end-start)/(end-sstart))

## LOF

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor(novelty=True, n_jobs=8).fit(data) # auto encoder에서 사용한거 그대로 사용.
lof_score = df[['MEASR_DTTM', 'STATUS']].copy()
lof_score['SCORE']= lof.score_samples(auto_df.iloc[:,1:-1])*-1

In [ ]:
# threshold 설정
import numpy as np
nboot = 100
alpha=0.0001 # 백분위 기준 이상치가 0.1%되게 threshold 설정한다는 의미
threshold_series = lof_score[lof_score['MEASR_DTTM']<train_test_split_point]['SCORE']

bootsam = np.random.choice(threshold_series, size=len(threshold_series) * nboot, replace=True, p=None)
bootsam.shape = (nboot, len(threshold_series))

lof_threshold  = np.mean(np.percentile(bootsam, q=(1 - alpha) * 100, axis=1))
lof_threshold

In [ ]:
plot_df = lof_score.copy()
threshold = lof_threshold
# threshold =0.12
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
train_test_split_point2 = train_test_split_point.timestamp()*1000
import plotly.express as px

# threshold 아래부분 sampling해서 그래프 용량 줄이기
if len(plot_df) > 100000:
    tmp = plot_df[plot_df['SCORE'] < threshold].sort_values(by='SCORE', ascending = False).copy()
    tmp = tmp[:100000]
    plot_df = pd.concat((tmp, plot_df[plot_df['SCORE']>=threshold]))
    plot_df = plot_df.sort_values('MEASR_DTTM')

plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
fig.add_vline(x=train_test_split_point2
                  , line_dash="dash"
                 , annotation_text = 'Test 구간'
    #               , line_color="red"
                 )
fig.add_hrect(
    y0=threshold, 
    y1=np.min(plot_df['이상 점수']), line_width=0, 
    annotation_text = '정상구간',
    fillcolor="green", opacity=0.2)
fig.show()
# test set에서 false 알람.
tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
print('test set에서 false 경고: ', len(tmp), 'test set에서 true 경고: ', len(tmp2))
print('train set에서 false 경고: ', len(tmp3))

## 15분 내 3번 경고시 알람

In [ ]:
import datetime
timedel = 15 # 몇분 이내에 3회이상 경고 떠야 알람 울릴 것인지
cutoff = 3 # 15분 이내에 몇개 이상일 때알람 울릴 것인지
alarm_df = lof_score.copy()
alarm_df['ALARM'] = False
for i in np.where(alarm_df['SCORE'] > threshold)[0]:
    tmp = alarm_df[(alarm_df['MEASR_DTTM']<alarm_df.iloc[i,0])&(alarm_df['MEASR_DTTM']>=alarm_df.iloc[i,0]-datetime.timedelta(minutes=timedel))]
    if len(tmp[tmp['SCORE']>threshold])>cutoff:
        alarm_df.iloc[i, -1] = True

tmp = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = alarm_df[(alarm_df['ALARM']==True)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]      
print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
print('train set에서 false 알람: ', len(tmp3))
start =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
if len(tmp2) >0:
    start = tmp2.iloc[0,0]
    print('가짜알람비율: ',len(tmp)/(len(tmp)+len(tmp2)))
else:
    print('알람없음.')
sstart =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
end = alarm_df[alarm_df['STATUS']=='전조'].iloc[-1,0]

print('최초알람시점 비율',(end-start)/(end-sstart))

## EllipticEnvelope

In [ ]:
from sklearn.covariance import EllipticEnvelope
ellip = EllipticEnvelope(random_state=0).fit(data) # auto encoder에서 사용한거 그대로 사용.
ellip_score = df[['MEASR_DTTM', 'STATUS']].copy()
ellip_score['SCORE']= ellip.score_samples(auto_df.iloc[:,1:-1])*-1

In [ ]:
# threshold 설정
import numpy as np
nboot = 100
alpha=0.0001 # 백분위 기준 이상치가 0.1%되게 threshold 설정한다는 의미
threshold_series = ellip_score[ellip_score['MEASR_DTTM']<train_test_split_point]['SCORE']

bootsam = np.random.choice(threshold_series, size=len(threshold_series) * nboot, replace=True, p=None)
bootsam.shape = (nboot, len(threshold_series))

ellip_threshold  = np.mean(np.percentile(bootsam, q=(1 - alpha) * 100, axis=1))
ellip_threshold

In [ ]:
plot_df = ellip_score.copy()
threshold = ellip_threshold
# threshold =0.12
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
train_test_split_point2 = train_test_split_point.timestamp()*1000
import plotly.express as px

# threshold 아래부분 sampling해서 그래프 용량 줄이기
if len(plot_df) > 100000:
    tmp = plot_df[plot_df['SCORE'] < threshold].sort_values(by='SCORE', ascending = False).copy()
    tmp = tmp[:100000]
    plot_df = pd.concat((tmp, plot_df[plot_df['SCORE']>=threshold]))
    plot_df = plot_df.sort_values('MEASR_DTTM')

plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
fig.add_vline(x=train_test_split_point2
                  , line_dash="dash"
                 , annotation_text = 'Test 구간'
    #               , line_color="red"
                 )
fig.add_hrect(
    y0=threshold, 
    y1=np.min(plot_df['이상 점수']), line_width=0, 
    annotation_text = '정상구간',
    fillcolor="green", opacity=0.2)
fig.show()
# test set에서 false 알람.
tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
print('test set에서 false 경고: ', len(tmp), 'test set에서 true 경고: ', len(tmp2))
print('train set에서 false 경고: ', len(tmp3))

## 15분 내 3번 경고시 알람

In [ ]:
import datetime
timedel = 15 # 몇분 이내에 3회이상 경고 떠야 알람 울릴 것인지
cutoff = 3 # 15분 이내에 몇개 이상일 때알람 울릴 것인지
alarm_df = ellip_score.copy()
alarm_df['ALARM'] = False
for i in np.where(alarm_df['SCORE'] > threshold)[0]:
    tmp = alarm_df[(alarm_df['MEASR_DTTM']<alarm_df.iloc[i,0])&(alarm_df['MEASR_DTTM']>=alarm_df.iloc[i,0]-datetime.timedelta(minutes=timedel))]
    if len(tmp[tmp['SCORE']>threshold])>cutoff:
        alarm_df.iloc[i, -1] = True

tmp = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = alarm_df[(alarm_df['ALARM']==True)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]      
print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
print('train set에서 false 알람: ', len(tmp3))
start =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
if len(tmp2) >0:
    start = tmp2.iloc[0,0]
    print('가짜알람비율: ',len(tmp)/(len(tmp)+len(tmp2)))
else:
    print('알람없음.')
sstart =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
end = alarm_df[alarm_df['STATUS']=='전조'].iloc[-1,0]

print('최초알람시점 비율',(end-start)/(end-sstart))

## Auto Encoder(MLP)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
from tensorflow.keras import backend as K
checkpoint_path = 'best_weights2.h5'

scaler = MinMaxScaler()
data = train_data.iloc[:,1:-1].copy()
data[:] = scaler.fit_transform(data[:])

shape = data.shape
x1 = Input(shape=(shape[1],))
x = Dense(30, activation='relu')(x1)
x = Dropout(0.2)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(10, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(30, activation='relu')(x1)
x = Dropout(0.2)(x)
x2 = Dense(shape[1], activation='sigmoid')(x)

model = Model(x1, x2)

model.compile(optimizer='adam', loss='mse')

es = EarlyStopping(patience=10, mode='auto', monitor = 'val_loss')
mc = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True, verbose=1)


model.fit(data, data, epochs=100000, validation_split= 0.1, callbacks=[es, mc], batch_size=128, verbose=1) 
model.load_weights(checkpoint_path)


## 이미 오토인코더학습 했고 불러오기만 원할 때

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.models import Model
# from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
# from tensorflow.keras import backend as K
# checkpoint_path = 'best_weights.h5'

# scaler = MinMaxScaler()
# data = train_data.iloc[:,1:-1].copy()
# data[:] = scaler.fit_transform(data[:])

# shape = data.shape
# x1 = Input(shape=(shape[1],))
# x = Lambda(lambda x: K.expand_dims(x,axis=2))(x1)
# x = Conv1D(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1D(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1DTranspose(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=1, kernel_size=7, padding = 'same')(x)
# x = Flatten()(x)
# x2 = Dense(shape[1], activation='sigmoid')(x)

# model = Model(x1, x2)
# model.load_weights(checkpoint_path)

In [ ]:
auto_df = df.copy()
auto_df.iloc[:,1:-1] = scaler.transform(auto_df.iloc[:,1:-1])

In [ ]:
pred = model.predict(auto_df.iloc[:,1:-1])
score = np.mean(np.abs(pred - auto_df.iloc[:,1:-1]), axis=1)

auto_score = df[['MEASR_DTTM', 'STATUS']].copy()
auto_score['SCORE']= score

In [ ]:
# threshold 설정
import numpy as np
nboot = 100
alpha=0.0001 # 백분위 기준 이상치가 0.1%되게 threshold 설정한다는 의미
threshold_series = auto_score[auto_score['MEASR_DTTM']<train_test_split_point]['SCORE']

bootsam = np.random.choice(threshold_series, size=len(threshold_series) * nboot, replace=True, p=None)
bootsam.shape = (nboot, len(threshold_series))

auto_threshold  = np.mean(np.percentile(bootsam, q=(1 - alpha) * 100, axis=1))
auto_threshold

In [ ]:
threshold = auto_threshold
# threshold =0.12
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
train_test_split_point2 = train_test_split_point.timestamp()*1000
import plotly.express as px
plot_df = auto_score.copy()
# threshold 아래부분 sampling해서 그래프 용량 줄이기
if len(plot_df) > 100000:
    tmp = plot_df[plot_df['SCORE'] < threshold].sort_values(by='SCORE', ascending = False).copy()
    tmp = tmp[:100000]
    plot_df = pd.concat((tmp, plot_df[plot_df['SCORE']>=threshold]))
    plot_df = plot_df.sort_values('MEASR_DTTM')

plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
fig.add_vline(x=train_test_split_point2
                  , line_dash="dash"
                 , annotation_text = 'Test 구간'
    #               , line_color="red"
                 )
fig.add_hrect(
    y0=threshold, 
    y1=np.min(plot_df['이상 점수']), line_width=0, 
    annotation_text = '정상구간',
    fillcolor="green", opacity=0.2)
fig.show()
# test set에서 false 알람.
tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
print('test set에서 false 경고: ', len(tmp), 'test set에서 true 경고: ', len(tmp2))
print('train set에서 false 경고: ', len(tmp3))

## 15분 내 3번 경고시 알람

In [ ]:
import datetime
timedel = 15 # 몇분 이내에 3회이상 경고 떠야 알람 울릴 것인지
cutoff = 3 # 15분 이내에 몇개 이상일 때알람 울릴 것인지
alarm_df = auto_score.copy()
alarm_df['ALARM'] = False
for i in np.where(alarm_df['SCORE'] > threshold)[0]:
    tmp = alarm_df[(alarm_df['MEASR_DTTM']<alarm_df.iloc[i,0])&(alarm_df['MEASR_DTTM']>=alarm_df.iloc[i,0]-datetime.timedelta(minutes=timedel))]
    if len(tmp[tmp['SCORE']>threshold])>cutoff:
        alarm_df.iloc[i, -1] = True

tmp = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = alarm_df[(alarm_df['ALARM']==True)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]      
print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
print('train set에서 false 알람: ', len(tmp3))
start =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
if len(tmp2) >0:
    start = tmp2.iloc[0,0]
    print('가짜알람비율: ',len(tmp)/(len(tmp)+len(tmp2)))
else:
    print('알람없음.')
sstart =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
end = alarm_df[alarm_df['STATUS']=='전조'].iloc[-1,0]

print('최초알람시점 비율',(end-start)/(end-sstart))

## Y만 사용한 오토인코더(MLP)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
from tensorflow.keras import backend as K
checkpoint_path = 'best_weights_Y2.h5'

scaler = MinMaxScaler()
data2 = train_data.iloc[:,11:21].copy()
data2[:] = scaler.fit_transform(data2[:])

shape = data2.shape
x1 = Input(shape=(shape[1],))
x = Dense(30, activation='relu')(x1)
x = Dropout(0.2)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(10, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(30, activation='relu')(x1)
x = Dropout(0.2)(x)
x2 = Dense(shape[1], activation='sigmoid')(x)

model2 = Model(x1, x2)

model2.compile(optimizer='adam', loss='mse')

es = EarlyStopping(patience=10, mode='auto', monitor = 'val_loss')
mc = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True, verbose=1)


model2.fit(data2, data2, epochs=100000, validation_split= 0.1, callbacks=[es, mc], batch_size=128, verbose=1) 
model2.load_weights(checkpoint_path)


## 이미 오토인코더학습 했고 불러오기만 원할 때

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.models import Model
# from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
# from tensorflow.keras import backend as K
# checkpoint_path = 'best_weights_Y.h5'

# scaler = MinMaxScaler()
# data2 = train_data.iloc[:,11:21].copy()
# data2[:] = scaler.fit_transform(data2[:])

# shape = data2.shape
# x1 = Input(shape=(shape[1],))
# x = Lambda(lambda x: K.expand_dims(x,axis=2))(x1)
# x = Conv1D(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1D(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1DTranspose(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=1, kernel_size=7, padding = 'same')(x)
# x = Flatten()(x)
# x2 = Dense(shape[1], activation='sigmoid')(x)

# model2 = Model(x1, x2)
# model2.load_weights(checkpoint_path)

In [ ]:
auto_df2 = df.copy()
auto_df2.iloc[:,11:21] = scaler.transform(auto_df2.iloc[:,11:21])

In [ ]:
pred = model2.predict(auto_df2.iloc[:,11:21])
score = np.mean(np.abs(pred - auto_df2.iloc[:,11:21]), axis=1)

auto_score2 = df[['MEASR_DTTM', 'STATUS']].copy()
auto_score2['SCORE']= score

In [ ]:
# threshold 설정
import numpy as np
nboot = 100
alpha=0.0001 # 백분위 기준 이상치가 0.1%되게 threshold 설정한다는 의미
threshold_series = auto_score2[auto_score2['MEASR_DTTM']<train_test_split_point]['SCORE']

bootsam = np.random.choice(threshold_series, size=len(threshold_series) * nboot, replace=True, p=None)
bootsam.shape = (nboot, len(threshold_series))

auto_threshold2  = np.mean(np.percentile(bootsam, q=(1 - alpha) * 100, axis=1))
auto_threshold2

In [ ]:
threshold = auto_threshold2
# threshold =0.12
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
train_test_split_point2 = train_test_split_point.timestamp()*1000
import plotly.express as px
plot_df = auto_score2.copy()
# threshold 아래부분 sampling해서 그래프 용량 줄이기
if len(plot_df) > 100000:
    tmp = plot_df[plot_df['SCORE'] < threshold].sort_values(by='SCORE', ascending = False).copy()
    tmp = tmp[:100000]
    plot_df = pd.concat((tmp, plot_df[plot_df['SCORE']>=threshold]))
    plot_df = plot_df.sort_values('MEASR_DTTM')

plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
fig.add_vline(x=train_test_split_point2
                  , line_dash="dash"
                 , annotation_text = 'Test 구간'
    #               , line_color="red"
                 )
fig.add_hrect(
    y0=threshold, 
    y1=np.min(plot_df['이상 점수']), line_width=0, 
    annotation_text = '정상구간',
    fillcolor="green", opacity=0.2)
fig.show()
# test set에서 false 알람.
tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
print('test set에서 false 경고: ', len(tmp), 'test set에서 true 경고: ', len(tmp2))
print('train set에서 false 경고: ', len(tmp3))

## 15분 내 3번 경고시 알람

In [ ]:
import datetime
timedel = 15 # 몇분 이내에 3회이상 경고 떠야 알람 울릴 것인지
cutoff = 3 # 15분 이내에 몇개 이상일 때알람 울릴 것인지
alarm_df = auto_score2.copy()
alarm_df['ALARM'] = False
for i in np.where(alarm_df['SCORE'] > threshold)[0]:
    tmp = alarm_df[(alarm_df['MEASR_DTTM']<alarm_df.iloc[i,0])&(alarm_df['MEASR_DTTM']>=alarm_df.iloc[i,0]-datetime.timedelta(minutes=timedel))]
    if len(tmp[tmp['SCORE']>threshold])>cutoff:
        alarm_df.iloc[i, -1] = True

tmp = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = alarm_df[(alarm_df['ALARM']==True)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]      
print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
print('train set에서 false 알람: ', len(tmp3))
start =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
if len(tmp2) >0:
    start = tmp2.iloc[0,0]
    print('가짜알람비율: ',len(tmp)/(len(tmp)+len(tmp2)))
else:
    print('알람없음.')
sstart =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
end = alarm_df[alarm_df['STATUS']=='전조'].iloc[-1,0]

print('최초알람시점 비율',(end-start)/(end-sstart))

## X만 사용한 오토인코더(MLP)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
from tensorflow.keras import backend as K
checkpoint_path = 'best_weights_Y2.h5'

scaler = MinMaxScaler()
data2 = train_data.iloc[:,1:11].copy()
data2[:] = scaler.fit_transform(data2[:])

shape = data2.shape
x1 = Input(shape=(shape[1],))
x = Dense(30, activation='relu')(x1)
x = Dropout(0.2)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(10, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(30, activation='relu')(x1)
x = Dropout(0.2)(x)
x2 = Dense(shape[1], activation='sigmoid')(x)

model2 = Model(x1, x2)

model2.compile(optimizer='adam', loss='mse')

es = EarlyStopping(patience=10, mode='auto', monitor = 'val_loss')
mc = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True, verbose=1)


model2.fit(data2, data2, epochs=100000, validation_split= 0.1, callbacks=[es, mc], batch_size=128, verbose=1) 
model2.load_weights(checkpoint_path)


## 이미 오토인코더학습 했고 불러오기만 원할 때

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.models import Model
# from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
# from tensorflow.keras import backend as K
# checkpoint_path = 'best_weights_Y.h5'

# scaler = MinMaxScaler()
# data2 = train_data.iloc[:,11:21].copy()
# data2[:] = scaler.fit_transform(data2[:])

# shape = data2.shape
# x1 = Input(shape=(shape[1],))
# x = Lambda(lambda x: K.expand_dims(x,axis=2))(x1)
# x = Conv1D(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1D(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1DTranspose(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=1, kernel_size=7, padding = 'same')(x)
# x = Flatten()(x)
# x2 = Dense(shape[1], activation='sigmoid')(x)

# model2 = Model(x1, x2)
# model2.load_weights(checkpoint_path)

In [ ]:
auto_df2 = df.copy()
auto_df2.iloc[:,1:11] = scaler.transform(auto_df2.iloc[:,1:11])

In [ ]:
pred = model2.predict(auto_df2.iloc[:,1:11])
score = np.mean(np.abs(pred - auto_df2.iloc[:,1:11]), axis=1)

auto_score3 = df[['MEASR_DTTM', 'STATUS']].copy()
auto_score3['SCORE']= score

In [ ]:
# threshold 설정
import numpy as np
nboot = 100
alpha=0.0001 # 백분위 기준 이상치가 0.1%되게 threshold 설정한다는 의미
threshold_series = auto_score3[auto_score3['MEASR_DTTM']<train_test_split_point]['SCORE']

bootsam = np.random.choice(threshold_series, size=len(threshold_series) * nboot, replace=True, p=None)
bootsam.shape = (nboot, len(threshold_series))

auto_threshold3  = np.mean(np.percentile(bootsam, q=(1 - alpha) * 100, axis=1))
auto_threshold3

In [ ]:
threshold = auto_threshold3
# threshold =0.12
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
train_test_split_point2 = train_test_split_point.timestamp()*1000
import plotly.express as px
plot_df = auto_score3.copy()
# threshold 아래부분 sampling해서 그래프 용량 줄이기
if len(plot_df) > 100000:
    tmp = plot_df[plot_df['SCORE'] < threshold].sort_values(by='SCORE', ascending = False).copy()
    tmp = tmp[:100000]
    plot_df = pd.concat((tmp, plot_df[plot_df['SCORE']>=threshold]))
    plot_df = plot_df.sort_values('MEASR_DTTM')

plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
fig.add_vline(x=train_test_split_point2
                  , line_dash="dash"
                 , annotation_text = 'Test 구간'
    #               , line_color="red"
                 )
fig.add_hrect(
    y0=threshold, 
    y1=np.min(plot_df['이상 점수']), line_width=0, 
    annotation_text = '정상구간',
    fillcolor="green", opacity=0.2)
fig.show()
# test set에서 false 알람.
tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
print('test set에서 false 경고: ', len(tmp), 'test set에서 true 경고: ', len(tmp2))
print('train set에서 false 경고: ', len(tmp3))

## 15분 내 3번 경고시 알람

In [ ]:
import datetime
timedel = 15 # 몇분 이내에 3회이상 경고 떠야 알람 울릴 것인지
cutoff = 3 # 15분 이내에 몇개 이상일 때알람 울릴 것인지
alarm_df = auto_score3.copy()
alarm_df['ALARM'] = False
for i in np.where(alarm_df['SCORE'] > threshold)[0]:
    tmp = alarm_df[(alarm_df['MEASR_DTTM']<alarm_df.iloc[i,0])&(alarm_df['MEASR_DTTM']>=alarm_df.iloc[i,0]-datetime.timedelta(minutes=timedel))]
    if len(tmp[tmp['SCORE']>threshold])>cutoff:
        alarm_df.iloc[i, -1] = True

tmp = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = alarm_df[(alarm_df['ALARM']==True)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]      
print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
print('train set에서 false 알람: ', len(tmp3))
start =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
if len(tmp2) >0:
    start = tmp2.iloc[0,0]
    print('가짜알람비율: ',len(tmp)/(len(tmp)+len(tmp2)))
else:
    print('알람없음.')
sstart =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
end = alarm_df[alarm_df['STATUS']=='전조'].iloc[-1,0]

print('최초알람시점 비율',(end-start)/(end-sstart))

## Z만 사용한 오토인코더(MLP)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
from tensorflow.keras import backend as K
checkpoint_path = 'best_weights_Y2.h5'

scaler = MinMaxScaler()
data2 = train_data.iloc[:,21:31].copy()
data2[:] = scaler.fit_transform(data2[:])

shape = data2.shape
x1 = Input(shape=(shape[1],))
x = Dense(30, activation='relu')(x1)
x = Dropout(0.2)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(10, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(30, activation='relu')(x1)
x = Dropout(0.2)(x)
x2 = Dense(shape[1], activation='sigmoid')(x)

model2 = Model(x1, x2)

model2.compile(optimizer='adam', loss='mse')

es = EarlyStopping(patience=10, mode='auto', monitor = 'val_loss')
mc = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True, verbose=1)


model2.fit(data2, data2, epochs=100000, validation_split= 0.1, callbacks=[es, mc], batch_size=128, verbose=1) 
model2.load_weights(checkpoint_path)


## 이미 오토인코더학습 했고 불러오기만 원할 때

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.models import Model
# from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
# from tensorflow.keras import backend as K
# checkpoint_path = 'best_weights_Y.h5'

# scaler = MinMaxScaler()
# data2 = train_data.iloc[:,11:21].copy()
# data2[:] = scaler.fit_transform(data2[:])

# shape = data2.shape
# x1 = Input(shape=(shape[1],))
# x = Lambda(lambda x: K.expand_dims(x,axis=2))(x1)
# x = Conv1D(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1D(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1DTranspose(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=1, kernel_size=7, padding = 'same')(x)
# x = Flatten()(x)
# x2 = Dense(shape[1], activation='sigmoid')(x)

# model2 = Model(x1, x2)
# model2.load_weights(checkpoint_path)

In [ ]:
auto_df2 = df.copy()
auto_df2.iloc[:,21:31] = scaler.transform(auto_df2.iloc[:,21:31])

In [ ]:
pred = model2.predict(auto_df2.iloc[:,21:31])
score = np.mean(np.abs(pred - auto_df2.iloc[:,21:31]), axis=1)

auto_score4 = df[['MEASR_DTTM', 'STATUS']].copy()
auto_score4['SCORE']= score

In [ ]:
# threshold 설정
import numpy as np
nboot = 100
alpha=0.0001 # 백분위 기준 이상치가 0.1%되게 threshold 설정한다는 의미
threshold_series = auto_score4[auto_score4['MEASR_DTTM']<train_test_split_point]['SCORE']

bootsam = np.random.choice(threshold_series, size=len(threshold_series) * nboot, replace=True, p=None)
bootsam.shape = (nboot, len(threshold_series))

auto_threshold4  = np.mean(np.percentile(bootsam, q=(1 - alpha) * 100, axis=1))
auto_threshold4

In [ ]:
threshold = auto_threshold4
# threshold =0.12
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
train_test_split_point2 = train_test_split_point.timestamp()*1000
import plotly.express as px
plot_df = auto_score4.copy()
# threshold 아래부분 sampling해서 그래프 용량 줄이기
if len(plot_df) > 100000:
    tmp = plot_df[plot_df['SCORE'] < threshold].sort_values(by='SCORE', ascending = False).copy()
    tmp = tmp[:100000]
    plot_df = pd.concat((tmp, plot_df[plot_df['SCORE']>=threshold]))
    plot_df = plot_df.sort_values('MEASR_DTTM')

plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
fig.add_vline(x=train_test_split_point2
                  , line_dash="dash"
                 , annotation_text = 'Test 구간'
    #               , line_color="red"
                 )
fig.add_hrect(
    y0=threshold, 
    y1=np.min(plot_df['이상 점수']), line_width=0, 
    annotation_text = '정상구간',
    fillcolor="green", opacity=0.2)
fig.show()
# test set에서 false 알람.
tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
print('test set에서 false 경고: ', len(tmp), 'test set에서 true 경고: ', len(tmp2))
print('train set에서 false 경고: ', len(tmp3))

## 15분 내 3번 경고시 알람

In [ ]:
import datetime
timedel = 15 # 몇분 이내에 3회이상 경고 떠야 알람 울릴 것인지
cutoff = 3 # 15분 이내에 몇개 이상일 때알람 울릴 것인지
alarm_df = auto_score4.copy()
alarm_df['ALARM'] = False
for i in np.where(alarm_df['SCORE'] > threshold)[0]:
    tmp = alarm_df[(alarm_df['MEASR_DTTM']<alarm_df.iloc[i,0])&(alarm_df['MEASR_DTTM']>=alarm_df.iloc[i,0]-datetime.timedelta(minutes=timedel))]
    if len(tmp[tmp['SCORE']>threshold])>cutoff:
        alarm_df.iloc[i, -1] = True

tmp = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = alarm_df[(alarm_df['ALARM']==True)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]      
print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
print('train set에서 false 알람: ', len(tmp3))
start =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
if len(tmp2) >0:
    start = tmp2.iloc[0,0]
    print('가짜알람비율: ',len(tmp)/(len(tmp)+len(tmp2)))
else:
    print('알람없음.')
sstart =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
end = alarm_df[alarm_df['STATUS']=='전조'].iloc[-1,0]

print('최초알람시점 비율',(end-start)/(end-sstart))

## Y, Z만 사용한 오토인코더(MLP)

In [ ]:
train_data.iloc[:,1:31]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
from tensorflow.keras import backend as K
checkpoint_path = 'best_weights_Y2.h5'

scaler = MinMaxScaler()
data2 = train_data.iloc[:,11:31].copy()
data2[:] = scaler.fit_transform(data2[:])

shape = data2.shape
x1 = Input(shape=(shape[1],))
x = Dense(30, activation='relu')(x1)
x = Dropout(0.2)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(10, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(30, activation='relu')(x1)
x = Dropout(0.2)(x)
x2 = Dense(shape[1], activation='sigmoid')(x)

model2 = Model(x1, x2)

model2.compile(optimizer='adam', loss='mse')

es = EarlyStopping(patience=10, mode='auto', monitor = 'val_loss')
mc = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True, verbose=1)


model2.fit(data2, data2, epochs=100000, validation_split= 0.1, callbacks=[es, mc], batch_size=128, verbose=1) 
model2.load_weights(checkpoint_path)


## 이미 오토인코더학습 했고 불러오기만 원할 때

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.models import Model
# from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
# from tensorflow.keras import backend as K
# checkpoint_path = 'best_weights_Y.h5'

# scaler = MinMaxScaler()
# data2 = train_data.iloc[:,11:21].copy()
# data2[:] = scaler.fit_transform(data2[:])

# shape = data2.shape
# x1 = Input(shape=(shape[1],))
# x = Lambda(lambda x: K.expand_dims(x,axis=2))(x1)
# x = Conv1D(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1D(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1DTranspose(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=1, kernel_size=7, padding = 'same')(x)
# x = Flatten()(x)
# x2 = Dense(shape[1], activation='sigmoid')(x)

# model2 = Model(x1, x2)
# model2.load_weights(checkpoint_path)

In [ ]:
auto_df2 = df.copy()
auto_df2.iloc[:,11:31] = scaler.transform(auto_df2.iloc[:,11:31])

In [ ]:
pred = model2.predict(auto_df2.iloc[:,11:31])
score = np.mean(np.abs(pred - auto_df2.iloc[:,11:31]), axis=1)

auto_score4 = df[['MEASR_DTTM', 'STATUS']].copy()
auto_score4['SCORE']= score

In [ ]:
# threshold 설정
import numpy as np
nboot = 100
alpha=0.0001 # 백분위 기준 이상치가 0.1%되게 threshold 설정한다는 의미
threshold_series = auto_score4[auto_score4['MEASR_DTTM']<train_test_split_point]['SCORE']

bootsam = np.random.choice(threshold_series, size=len(threshold_series) * nboot, replace=True, p=None)
bootsam.shape = (nboot, len(threshold_series))

auto_threshold4  = np.mean(np.percentile(bootsam, q=(1 - alpha) * 100, axis=1))
auto_threshold4

In [ ]:
threshold = auto_threshold4
# threshold =0.12
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
train_test_split_point2 = train_test_split_point.timestamp()*1000
import plotly.express as px
plot_df = auto_score4.copy()
# threshold 아래부분 sampling해서 그래프 용량 줄이기
if len(plot_df) > 100000:
    tmp = plot_df[plot_df['SCORE'] < threshold].sort_values(by='SCORE', ascending = False).copy()
    tmp = tmp[:100000]
    plot_df = pd.concat((tmp, plot_df[plot_df['SCORE']>=threshold]))
    plot_df = plot_df.sort_values('MEASR_DTTM')

plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
fig.add_vline(x=train_test_split_point2
                  , line_dash="dash"
                 , annotation_text = 'Test 구간'
    #               , line_color="red"
                 )
fig.add_hrect(
    y0=threshold, 
    y1=np.min(plot_df['이상 점수']), line_width=0, 
    annotation_text = '정상구간',
    fillcolor="green", opacity=0.2)
fig.show()
# test set에서 false 알람.
tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
print('test set에서 false 경고: ', len(tmp), 'test set에서 true 경고: ', len(tmp2))
print('train set에서 false 경고: ', len(tmp3))

## 15분 내 3번 경고시 알람

In [ ]:
import datetime
timedel = 15 # 몇분 이내에 3회이상 경고 떠야 알람 울릴 것인지
cutoff = 3 # 15분 이내에 몇개 이상일 때알람 울릴 것인지
alarm_df = auto_score4.copy()
alarm_df['ALARM'] = False
for i in np.where(alarm_df['SCORE'] > threshold)[0]:
    tmp = alarm_df[(alarm_df['MEASR_DTTM']<alarm_df.iloc[i,0])&(alarm_df['MEASR_DTTM']>=alarm_df.iloc[i,0]-datetime.timedelta(minutes=timedel))]
    if len(tmp[tmp['SCORE']>threshold])>cutoff:
        alarm_df.iloc[i, -1] = True

tmp = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = alarm_df[(alarm_df['ALARM']==True)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]      
print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
print('train set에서 false 알람: ', len(tmp3))
start =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
if len(tmp2) >0:
    start = tmp2.iloc[0,0]
    print('가짜알람비율: ',len(tmp)/(len(tmp)+len(tmp2)))
else:
    print('알람없음.')
sstart =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
end = alarm_df[alarm_df['STATUS']=='전조'].iloc[-1,0]

print('최초알람시점 비율',(end-start)/(end-sstart))

## Y, Z만 사용한 오토인코더(MLP)

In [ ]:
train_data.iloc[:,1:31]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
from tensorflow.keras import backend as K
checkpoint_path = 'best_weights_Y2.h5'

scaler = MinMaxScaler()
data2 = train_data.iloc[:,1:31].copy()
data2[:] = scaler.fit_transform(data2[:])

shape = data2.shape
x1 = Input(shape=(shape[1],))
x = Dense(30, activation='relu')(x1)
x = Dropout(0.2)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(10, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(30, activation='relu')(x1)
x = Dropout(0.2)(x)
x2 = Dense(shape[1], activation='sigmoid')(x)

model2 = Model(x1, x2)

model2.compile(optimizer='adam', loss='mse')

es = EarlyStopping(patience=10, mode='auto', monitor = 'val_loss')
mc = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True, verbose=1)


model2.fit(data2, data2, epochs=100000, validation_split= 0.1, callbacks=[es, mc], batch_size=128, verbose=1) 
model2.load_weights(checkpoint_path)


## 이미 오토인코더학습 했고 불러오기만 원할 때

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.models import Model
# from keras.layers import Dense, Dropout, Conv1D, Input, Conv1DTranspose, Lambda, Flatten
# from tensorflow.keras import backend as K
# checkpoint_path = 'best_weights_Y.h5'

# scaler = MinMaxScaler()
# data2 = train_data.iloc[:,11:21].copy()
# data2[:] = scaler.fit_transform(data2[:])

# shape = data2.shape
# x1 = Input(shape=(shape[1],))
# x = Lambda(lambda x: K.expand_dims(x,axis=2))(x1)
# x = Conv1D(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1D(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=16, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Conv1DTranspose(filters=32, kernel_size=7, padding = 'same', strides=2, activation='relu')(x)
# x = Conv1DTranspose(filters=1, kernel_size=7, padding = 'same')(x)
# x = Flatten()(x)
# x2 = Dense(shape[1], activation='sigmoid')(x)

# model2 = Model(x1, x2)
# model2.load_weights(checkpoint_path)

In [ ]:
auto_df2 = df.copy()
auto_df2.iloc[:,1:31] = scaler.transform(auto_df2.iloc[:,1:31])

In [ ]:
pred = model2.predict(auto_df2.iloc[:,1:31])
score = np.mean(np.abs(pred - auto_df2.iloc[:,1:31]), axis=1)

auto_score4 = df[['MEASR_DTTM', 'STATUS']].copy()
auto_score4['SCORE']= score

In [ ]:
# threshold 설정
import numpy as np
nboot = 100
alpha=0.0001 # 백분위 기준 이상치가 0.1%되게 threshold 설정한다는 의미
threshold_series = auto_score4[auto_score4['MEASR_DTTM']<train_test_split_point]['SCORE']

bootsam = np.random.choice(threshold_series, size=len(threshold_series) * nboot, replace=True, p=None)
bootsam.shape = (nboot, len(threshold_series))

auto_threshold4  = np.mean(np.percentile(bootsam, q=(1 - alpha) * 100, axis=1))
auto_threshold4

In [ ]:
threshold = auto_threshold4
# threshold =0.12
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
train_test_split_point2 = train_test_split_point.timestamp()*1000
import plotly.express as px
plot_df = auto_score4.copy()
# threshold 아래부분 sampling해서 그래프 용량 줄이기
if len(plot_df) > 100000:
    tmp = plot_df[plot_df['SCORE'] < threshold].sort_values(by='SCORE', ascending = False).copy()
    tmp = tmp[:100000]
    plot_df = pd.concat((tmp, plot_df[plot_df['SCORE']>=threshold]))
    plot_df = plot_df.sort_values('MEASR_DTTM')

plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
fig.add_vline(x=train_test_split_point2
                  , line_dash="dash"
                 , annotation_text = 'Test 구간'
    #               , line_color="red"
                 )
fig.add_hrect(
    y0=threshold, 
    y1=np.min(plot_df['이상 점수']), line_width=0, 
    annotation_text = '정상구간',
    fillcolor="green", opacity=0.2)
fig.show()
# test set에서 false 알람.
tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
print('test set에서 false 경고: ', len(tmp), 'test set에서 true 경고: ', len(tmp2))
print('train set에서 false 경고: ', len(tmp3))

## 15분 내 3번 경고시 알람

In [ ]:
import datetime
timedel = 15 # 몇분 이내에 3회이상 경고 떠야 알람 울릴 것인지
cutoff = 3 # 15분 이내에 몇개 이상일 때알람 울릴 것인지
alarm_df = auto_score4.copy()
alarm_df['ALARM'] = False
for i in np.where(alarm_df['SCORE'] > threshold)[0]:
    tmp = alarm_df[(alarm_df['MEASR_DTTM']<alarm_df.iloc[i,0])&(alarm_df['MEASR_DTTM']>=alarm_df.iloc[i,0]-datetime.timedelta(minutes=timedel))]
    if len(tmp[tmp['SCORE']>threshold])>cutoff:
        alarm_df.iloc[i, -1] = True

tmp = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = alarm_df[(alarm_df['ALARM']==True)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]      
print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
print('train set에서 false 알람: ', len(tmp3))
start =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
if len(tmp2) >0:
    start = tmp2.iloc[0,0]
    print('가짜알람비율: ',len(tmp)/(len(tmp)+len(tmp2)))
else:
    print('알람없음.')
sstart =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
end = alarm_df[alarm_df['STATUS']=='전조'].iloc[-1,0]

print('최초알람시점 비율',(end-start)/(end-sstart))

## IsolationForest (minmax scale)

In [ ]:
isol3 = IsolationForest(random_state=0).fit(data) # auto encoder에서 사용한거 그대로 사용.
isol3_score = df[['MEASR_DTTM', 'STATUS']].copy()
isol3_score['SCORE']= isol3.score_samples(auto_df.iloc[:,1:-1])*-1

In [ ]:
# threshold 설정
import numpy as np
nboot = 100
alpha=0.0001 # 백분위 기준 이상치가 0.1%되게 threshold 설정한다는 의미
threshold_series = isol3_score['SCORE']

bootsam = np.random.choice(threshold_series, size=len(threshold_series) * nboot, replace=True, p=None)
bootsam.shape = (nboot, len(threshold_series))

isol3_threshold  = np.mean(np.percentile(bootsam, q=(1 - alpha) * 100, axis=1))
isol3_threshold

In [ ]:
plot_df = isol3_score.copy()
threshold = isol3_threshold
# threshold =0.12
train_test_split_point = pd.to_datetime('2021-08-27 00:00:00')
train_test_split_point2 = train_test_split_point.timestamp()*1000
import plotly.express as px

# threshold 아래부분 sampling해서 그래프 용량 줄이기
if len(plot_df) > 100000:
    tmp = plot_df[plot_df['SCORE'] < threshold].sort_values(by='SCORE', ascending = False).copy()
    tmp = tmp[:100000]
    plot_df = pd.concat((tmp, plot_df[plot_df['SCORE']>=threshold]))
    plot_df = plot_df.sort_values('MEASR_DTTM')

plot_df.rename(columns={'SCORE': '이상 점수'}, inplace = True)  
fig = px.scatter(plot_df, x='MEASR_DTTM', y='이상 점수', color='STATUS')
fig.add_hline(y=threshold, line_dash="dash", line_color = 'red', annotation_text = 'Threshold')
fig.add_vline(x=train_test_split_point2
                  , line_dash="dash"
                 , annotation_text = 'Test 구간'
    #               , line_color="red"
                 )
fig.add_hrect(
    y0=threshold, 
    y1=np.min(plot_df['이상 점수']), line_width=0, 
    annotation_text = '정상구간',
    fillcolor="green", opacity=0.2)
fig.show()
# test set에서 false 알람.
tmp = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = plot_df[(plot_df['이상 점수']>threshold)&(plot_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = plot_df[(plot_df['이상 점수']>threshold)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]
print('test set에서 false 경고: ', len(tmp), 'test set에서 true 경고: ', len(tmp2))
print('train set에서 false 경고: ', len(tmp3))

## 15분 내 3번 경고시 알람

In [ ]:
import datetime
timedel = 15 # 몇분 이내에 3회이상 경고 떠야 알람 울릴 것인지
cutoff = 3 # 15분 이내에 몇개 이상일 때알람 울릴 것인지
alarm_df = isol3_score.copy()
alarm_df['ALARM'] = False
for i in np.where(alarm_df['SCORE'] > threshold)[0]:
    tmp = alarm_df[(alarm_df['MEASR_DTTM']<alarm_df.iloc[i,0])&(alarm_df['MEASR_DTTM']>=alarm_df.iloc[i,0]-datetime.timedelta(minutes=timedel))]
    if len(tmp[tmp['SCORE']>threshold])>cutoff:
        alarm_df.iloc[i, -1] = True

tmp = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='정상')]
tmp = tmp[tmp['MEASR_DTTM']>train_test_split_point]
tmp2 = alarm_df[(alarm_df['ALARM']==True)&(alarm_df['STATUS']=='전조')]
tmp2 = tmp2[tmp2['MEASR_DTTM']>train_test_split_point]
tmp3 = alarm_df[(alarm_df['ALARM']==True)]
tmp3 = tmp3[tmp3['MEASR_DTTM']<train_test_split_point]      
print('test set에서 false 알람: ', len(tmp), 'test set에서 true 알람: ', len(tmp2))
print('train set에서 false 알람: ', len(tmp3))
start =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
if len(tmp2) >0:
    start = tmp2.iloc[0,0]
    print('가짜알람비율: ',len(tmp)/(len(tmp)+len(tmp2)))
else:
    print('알람없음.')
sstart =alarm_df[alarm_df['STATUS']=='전조'].iloc[0,0]
end = alarm_df[alarm_df['STATUS']=='전조'].iloc[-1,0]

print('최초알람시점 비율',(end-start)/(end-sstart))